# 18. Эмуляция контейнеров
Напишем свой аналог листа таблицы Excel. Нужно написать структуру данных Field, в которой доступ к значениям будет осуществляться по ключам. Ключом будет пара "буква" - "число", по аналогии с адресом ячейки в Excel. Возможные форматы обращения к одной и той же "ячейке" данных:

field = Field()

- field[1, 'a'] = 25
- field['a', 1] = 25
- field['a', '1'] = 25
- field['1', 'a'] = 25
- field['1a'] = 25
- field['a1'] = 25
- field[1, 'A'] = 25
- field['A', 1] = 25
- field['A', '1'] = 25
- field['1', 'A'] = 25
- field['1A'] = 25
- field['A1'] = 25

В этом списке каждая из этих строк записывает число 25 в ячейку с одним и тем же ключом. Соответственно, по любому из перечисленных ключей должно быть можно получить это число из объекта field. Также должны быть реализованы удаление элемента из структуры (через оператор del) и возможность использования оператора in, например:

- (1, 'a') in field: True
- "A1" in field: True
- ('D', '4') in field: False

Таким образом, выходит, что ключом структуры может быть либо кортеж, либо строка. При попытке получить или записать значение по ключу другого типа должно быть вызвано исключение TypeError. При некорректном значении строки или элементов кортежа нужно вызывать исключение ValueError. Корректными значениями будет считать одиночные буквы и неотрицательное целое число любой длины, т.е. правильные варианты ключей:

- А1
- А222543
- Z89

Неправильные варианты ключей:

- AA5
- Q2.5
- -6F
- A
- 27
- GG

Кроме вышеперчисленного, по объекту должно быть возможно итерироваться. При проходе циклом по объекту должны возвращаться значения, хранящиеся в нём. Порядок возврата значений не важен.

Если запрашивается правильный формат ячейки, но в нашем контейнере такого ключа нет, то нужно вернуть None. Например:

field = Field()

print(field["C5"] is None)

\> True

#### Примечания
В своем решении этого задания я использовал в качестве ключей хранимого словаря frozenset, а проверку на ValueError реализовал через регулярку. Также рекомендую проверку типов и преобразование поступившего ключа в тот вид, в котором он хранится "под капотом", вынести в отдельный метод и вызывать его из всех описываемых магических методов.

In [1]:
import re
class Field(dict):
    
    def key_trans_and_comp(self,key):
        comp_pat1 = re.compile(r'[a-zа-яё]\d+$')
        comp_pat2 = re.compile(r'\d+[a-zа-яё]$')
        if type(key) == tuple and len(key) == 2:
            key = list(key)
            for i in range(len(key)):
                if type(key[i]) == int:
                    key[i] = str(key[i])
                elif type(key[i]) == str:
                    pass
                else:
                    raise ValueError
            key = ''.join(key)
        if type(key) == str:
            key = key.lower()
        else:
            raise TypeError
        if comp_pat1.match(key):
            return key
        if comp_pat2.match(key):
            return key[::-1]
        else:
            raise ValueError
            
    def __getitem__(self, key):
        return super(Field, self).__getitem__(self.key_trans_and_comp(key))
    
    def __setitem__(self, key, value):
        super(Field, self).__setitem__(self.key_trans_and_comp(key), value)
        
    def __delitem__(self, key):
        super(Field, self).__delitem__(self.key_trans_and_comp(key))
    
    def __missing__(self, key):
        return None
    
    def __contains__(self, item):
        return self[item] != self.__missing__(1)
    
    def __iter__(self):
        for elem in self.values():
            yield elem

In [2]:
field = Field()
field['a', 1] = 25
field['a', 2] = 31
field['b', 1] = 99
field['b', 2] = 42
field['a', 1]

25

In [3]:
print(field["C5"] is None)

True


In [4]:
field['a', 1, 10, 11] = 25 #должна быть ошибка типа

TypeError: 

In [5]:
field['a', 2.5] = 25 #должна быть ошибка значения

ValueError: 

In [6]:
field['a2.5'] = 25 #должна быть ошибка значения

ValueError: 

In [7]:
print('a1' in field)
print('b2' in field)
print(('a', 1) in field)
print('c1' in field)


True
True
True
False


In [8]:
print(field)

{'a1': 25, 'a2': 31, 'b1': 99, 'b2': 42}


In [9]:
a = field.__iter__()
a.__next__()

25

In [10]:
print(a.__next__())
print(a.__next__())
print(a.__next__())

31
99
42


# 19. Доступ к атрибутам
Доработать класс `Field` так, чтобы вдобавок к реализованному функционалу появились следующие возможности:

`field = Field()`
- Запись значения в ячейку:
    - `field.a1 = 25` - эквивалентно `field['a1'] = 25`
    - `field.A1 = 25` - то же самое
- Получение значения:
`field['b', 2] = 100
field.b2
field.B2`

- Удаление значения:
`del field.a1`, `del field.A1` - эквивалентно `del field['a', 1]`

Таким образом, внутри класса `Field` методы работы с атрибутами должны работать с тем же объектом, в котором хранятся значения, обрабатываемые в методах `__setitem__`, `__getitem__`, `__delitem__`.

Кроме того, обычное присвоение и получение атрибутов (тех, которые не являются адресом ячейки данных нашего класса) должно производиться по стандартному алгоритму питоновских объектов, т.е. они должны храниться в словаре `__dict__` объекта.

`field = Field()
field.abcde = 125
field.__dict__['abcde'] == 125`

Для таких атрибутов также должны быть реализованы получение, присваивание и удаление значения.

In [11]:
import re
class Field(dict):
    
    def key_trans_and_comp(self,key):
        comp_pat1 = re.compile(r'[a-zа-яё]\d+$')
        comp_pat2 = re.compile(r'\d+[a-zа-яё]$')
        if type(key) == tuple and len(key) == 2:
            key = list(key)
            for i in range(len(key)):
                if type(key[i]) == int:
                    key[i] = str(key[i])
                elif type(key[i]) == str:
                    pass
                else:
                    raise ValueError
            key = ''.join(key)
        if type(key) == str:
            key = key.lower()
        else:
            raise TypeError
        if comp_pat1.match(key):
            return key
        if comp_pat2.match(key):
            return key[::-1]
        else:
            raise ValueError
            
    def name_comp(self, name):
        comp_pat = re.compile(r'[A-Za-zА-Яа-яёЁ]\d+$')
        if comp_pat.match(name):
            return True
        else:
            False        
            
    def __getitem__(self, key):
        return super(Field, self).__getitem__(self.key_trans_and_comp(key))
    
    def __setitem__(self, key, value):
        super(Field, self).__setitem__(self.key_trans_and_comp(key), value)
        self.__dict__.update(self)
        
    def __delitem__(self, key):
        super(Field, self).__delitem__(self.key_trans_and_comp(key))
        del self.__dict__[self.key_trans_and_comp(key)]
    
    def __missing__(self, key):
        return None
    
    def __contains__(self, item):
        return self[item] != self.__missing__(1)
    
    def __iter__(self):
        for elem in self.values():
            yield elem          
        
    def __setattr__(self, name, value):
        if self.name_comp(name):
            name = name.lower()
            self.update({name : value})
            self.__dict__.update(self)
        else:
            self.__dict__.update({name : value})
    
    def __delattr__(self, name):
        if self.name_comp(name):
            name = name.lower()
            del self[name]
        else:
            del self.__dict__[name]
        
    def __getattr__(self, name):
        if self.name_comp(name):
            name = name.lower()
            return self.__dict__[name]
        else:
            return self.__dict__[name]

In [12]:
field1 = Field()
field1['a', 1] = 25
field1['a', 2] = 31
field1['b', 1] = 99
field1['b', 2] = 42
field1['a', 1]

25

In [13]:
field1.__dict__

{'a1': 25, 'a2': 31, 'b1': 99, 'b2': 42}

In [14]:
field1.uiytuy = 12

In [15]:
field1.__dict__

{'a1': 25, 'a2': 31, 'b1': 99, 'b2': 42, 'uiytuy': 12}

In [16]:
field1

{'a1': 25, 'a2': 31, 'b1': 99, 'b2': 42}

In [17]:
field1.uiytuy

12

In [18]:
del field1.uiytuy

In [19]:
field1.__dict__

{'a1': 25, 'a2': 31, 'b1': 99, 'b2': 42}

In [20]:
field1

{'a1': 25, 'a2': 31, 'b1': 99, 'b2': 42}

In [21]:
field1.A1 = 10
field1.AAa22222 = 100

In [22]:
print(field1)
print(field1.__dict__)
print(field1.a1)
print(field1.A1)

{'a1': 10, 'a2': 31, 'b1': 99, 'b2': 42}
{'a1': 10, 'a2': 31, 'b1': 99, 'b2': 42, 'AAa22222': 100}
10
10


In [23]:
del field1['b', 1]
print(field1)
print(field1.__dict__)

{'a1': 10, 'a2': 31, 'b2': 42}
{'a1': 10, 'a2': 31, 'b2': 42, 'AAa22222': 100}


In [24]:
del field1.A1
print(field1)
print(field1.__dict__)

{'a2': 31, 'b2': 42}
{'a2': 31, 'b2': 42, 'AAa22222': 100}


# 20. Исключения
Напишем часть сервиса, который будет помогать бронировать переговорки в офисе. Для этого опишем класс `Booking` - его объекты будут содержать информацию о конкретном бронировании, а также вспомогательную функцию `create_booking`, которая будет создавать новый объект бронирования и записывать информацию о бронировании в базу данных бронирований через предоставляемое API. Возвращать она должна будет статус создания бронирования (получилось или переговорка уже занята) и информацию о брони в формате JSON. Ниже - подробности.

Класс `Booking` должен обладать следующим функционалом.

- конструктор должен принимать три аргумента в следующем порядке: название переговорки, datetime начала брони и datetime конца брони
- внутри конструктора, если datetime конца брони оказался раньше, чем datetime начала, нужно вызвать исключение ValueError

Также у объектов этого класса должны быть следующие поля (рекомендую сделать часть из них в виде проперти):

- `room_name`: название переговорки, полученное из конструктора
- `start`: datetime начала брони. Должна быть возможность назначить новое время начала уже созданной брони
- `end`: datetime конца брони. Должна быть возможность назначить новое время конца уже созданной брони
- `duration`: длительность бронирования в минутах (гарантируется, что длительность любой встречи кратна одной минуте, поэтому это должно быть целое число)
- `start_date`: дата начала брони в формате YYYY-MM-DD (строка)
- `end_date`: дата конца брони в формате YYYY-MM-DD (строка)
- `start_time`: время начала брони в формате HH-MM (строка)
- `end_time`: время конца брони в формате HH-MM (строка)

Функция create_booking должна обладать следующей сигнатурой:

`create_booking(room_name, start, end) -> str`,

где аргументы - это те же аргументы, которые принимает конструктор Booking, а выходная строка - это json определенного формата, который описан чуть ниже по тексту.

Будем считать, что взаимодействие с базой данных у нас уже описано нашим коллегой в соседнем файле api.py. В нем есть уже готовая функция `register_booking`, которая:

- принимает на вход объект класса Booking
- возвращает True, если бронирование получилось создать
- возвращает False, если мы пытаемся забронировать уже занятую в это время переговорку
- если такой переговорки не существует, вызывается KeyError

Таким образом, в том же файле, что и класс Booking, вам нужно описать фукнцию `create_booking`, которая:

- обладает сигнатурой create_booking(room_name, start, end) -> str, где аргументы - те же, что и в конструкторе Booking
- в самом начале своей работы выводит на экран текст: Начинаем создание бронирования
- внутри функции создается объект класса Booking, а также вызывается функция register_booking, которая принимает на вход созданный объект. Должны быть обработаны все случаи работы register_booking: True, False и KeyError. Сделать это поможет конструкция try-except
- перед выходом из функции должно выводиться на экран сообщение Заканчиваем создание бронирования. Это должно происходить в любом случае, даже если мы попытались создать бронирование с неверными датами и получили ValueError (см. описание класса Booking). Для этого рекомендую использовать блок finally, в котором описать этот print

Функция должна возвращать json-строку с ответом, в котором будут содержаться следующие поля:

- `created`: true/false, получилось ли забронировать комнату. Если возникло KeyError, то нужно здесь записать false
- `msg`: сообщение с пояснениями. Сообщение должно быть одним из следующих: Бронирование создано, Комната занята, Комната не найдена. Сообщение выбирается на основе того, что вернет функция `register_booking`
- `booking`: это бронирование в виде json-строки. Должны содержаться поля: room_name, duration, start_date, end_date, start_time, end_time.

#### Формат ввода
result = create_booking(
    
    "Вагнер",
    
    datetime.datetime(2022, 9, 1, 14),
    
    datetime.datetime(2022, 9, 1, 15, 15)

)

print(result)

#### Формат вывода
Вывод на экран:

Начинаем создание бронирования

Заканчиваем создание бронирования

Функция возвращает:

{
  
  "created": false,
  
  "msg": "Комната занята",
  
  "booking": {
    
    "room_name": "Вагнер",
    
    "start_date": "2022-09-01",
    
    "start_time": "14:00",
    
    "end_date": "2022-09-01",
    
    "end_time": "15:15",
    
    "duration": 75
  
  }

}

#### Примечания
Пример написания функции create_booking:

from api import register_booking

def create_booking(room_name, start, end):
    booking = Booking(........)
    try:
        result = register_booking(booking)
    except ....:
        ....

    return json.dumps(......)


In [25]:
import datetime
import json
class Booking:
    
    def __init__(self, room_name, start, end):
        if start > end:
            raise ValueError
        else:    
            self.room_name = room_name
            self.start = start
            self.end = end
    
    @property
    def duration(self):
        return (self.end - self.start).seconds // 60
    
    @property
    def start_date(self):
        return "{:04}-{:02}-{:02}".format(self.start.year, self.start.month, self.start.day)
    
    @property
    def end_date(self):
        return "{:04}-{:02}-{:02}".format(self.end.year, self.end.month, self.end.day)
    
    @property
    def start_time(self):
        return "{:02}:{:02}".format(self.start.hour, self.start.minute)
    
    @property
    def end_time(self):
        return "{:02}:{:02}".format(self.end.hour, self.end.minute)
    
def create_booking(room_name, start, end):
    print('Начинаем создание бронирования')
    booking = Booking(room_name, start, end)
    try:
        result = register_booking(booking)
        if result == True:
            msg = 'Бронирование создано'
        elif result == False:
            msg = 'Комната занята'
    except KeyError:
        result = False
        msg = 'Комната не найдена'
    finally:
        print('Заканчиваем создание бронирования')
    return json.dumps({"created": result,
                       "msg": msg,
                       "booking": {"room_name": booking.room_name,
                                   "start_date": booking.start_date,
                                   "start_time": booking.start_time,
                                   "end_date": booking.end_date,
                                   "end_time": booking.end_time,
                                   "duration": booking.duration}
                      })    

In [26]:
booking = Booking("Вагнер",datetime.datetime(2022, 9, 1, 14),datetime.datetime(2022, 9, 1, 15, 15))
print({"room_name": booking.room_name,
                                   "start_date": booking.start_date,
                                   "start_time": booking.start_time,
                                   "end_date": booking.end_date,
                                   "end_time": booking.end_time,
                                   "duration": booking.duration})

{'room_name': 'Вагнер', 'start_date': '2022-09-01', 'start_time': '14:00', 'end_date': '2022-09-01', 'end_time': '15:15', 'duration': 75}


In [27]:
(datetime.datetime(2022, 9, 1, 15, 15) - datetime.datetime(2022, 9, 1, 14)).seconds // 60

75

In [28]:
datetime.datetime(2022, 9, 1, 15, 15).minute

15